In [1]:
import pandas as pd
import sqlite3
from db import *


In [2]:
stats_list = [
            'HomePoss', 
            # 'AwayPoss',
            'HomeShots',
            'AwayShots',
            'HomeSonT',
            'AwaySonT',
            'HomeSoffT',
            'AwaySoffT',
            'HomeBS',
            'AwayBS',
            'HomeCor',
            'AwayCor',
            'HomeOff',
            'AwayOff',
            'HomeFoul',
            'AwayFoul',
            'HomeYellow',
            'AwayYellow',
            'HomeRed',
            'AwayRed',
            'HomePass',
            'AwayPass',
            'HomeAccPass',
            'AwayAccPass',
            'HomePassOff',
            'AwayPassOff',
            'HomeAccLongB',
            'AwayAccLongB',
            'HomeAccLongBpercent',
            'AwayAccLongBpercent',
            'HomeAccCross',
            'AwayAccCross',
            'HomeAccCrosspercent',
            'AwayAccCrosspercent',
            'HomeSuccDribb',
            'AwaySuccDribb',
            'HomeSuccDribbpercent',
            'AwaySuccDribbpercent',
            'HomeDuelsW',
            'AwayDuelsW',
            'HomeTackW',
            'AwayTackW',
            'HomeTackWpercent',
            'AwayTackWpercent',
            'HomeInt',
            'AwayInt',
            'HomeClear',
            'AwayClear'
            ]

home_stats_list = [
            'HomePoss',
            'HomeShots',
            'HomeSonT',
            # 'HomeSoffT',
            'HomeBS',
            'HomeCor',
            'HomeOff',
            'HomeFoul',
            'HomeYellow',
            'HomeRed',
            'HomePass',
            'HomeAccPass',
            'HomePassOff',
            'HomeAccLongB',
            'HomeAccLongBpercent',
            'HomeAccCross',
            'HomeAccCrosspercent',
            'HomeSuccDribb',
            'HomeSuccDribbpercent',
            'HomeDuelsW',
            'HomeTackW',
            'HomeTackWpercent',
            'HomeInt',
            'HomeClear'
            ]

away_stats_list = [
            # 'AwayPoss',
            'AwayShots',
            'AwaySonT',
            # 'AwaySoffT',
            'AwayBS',
            'AwayCor',
            'AwayOff',
            'AwayFoul',
            'AwayYellow',
            'AwayRed',
            'AwayPass',
            'AwayAccPass',
            'AwayPassOff',
            'AwayAccLongB',
            'AwayAccLongBpercent',
            'AwayAccCross',
            'AwayAccCrosspercent',
            'AwaySuccDribb',
            'AwaySuccDribbpercent',
            'AwayDuelsW',
            'AwayTackW',
            'AwayTackWpercent',
            'AwayInt',
            'AwayClear'
            ]

In [3]:
conn = sqlite3.connect('MatchDB.db')
df = pd.read_sql_query("SELECT * FROM Games", conn)

In [4]:
df = df.sort_values('date')

In [5]:
i = 1

for index, row in df.iterrows():

    match_id = int(row['match_id'])

    query = f"SELECT * FROM Expected_Games WHERE match_id = {match_id}"
    in_database = call_db(query)

    Home_xStats = {}
    Away_xStats = {}

    if row['match_id'] and not in_database:
        HomeTeam = row['HomeTeam']
        AwayTeam = row['AwayTeam']
        date = row['date']
        League = row['League']
        Country = row['Country']
        HGoals = row['HGoals']
        AGoals = row['AGoals']

        HGames = f"SELECT Count(HomeTeam) FROM Games WHERE HomeTeam = '{HomeTeam}' AND date <= '{date}'"
        AGames = f"SELECT Count(AwayTeam) FROM Games WHERE HomeTeam = '{AwayTeam}' AND date <= '{date}'"

        HGames = get_value(HGames,)
        AGames = get_value(AGames,)


        

        league_stats = {}
        home_stats = {}
        away_stats = {}

        try:
            if HGames > 5 and AGames > 5:
                
                for stat in stats_list:
                    #Goals scoerd
                    home_avr = f"SELECT AVG({stat}) FROM Games WHERE HomeTeam = '{HomeTeam}' AND League = '{League}' AND {stat} IS NOT NULL AND date < '{date}'"
                    home_avr = get_value(home_avr,)
                    home_avr = round(home_avr, 2)
                    
                    home_stats[stat] = home_avr

                    

                for stat in stats_list:
                #Goals scoerd
                    away_avr = f"SELECT AVG({stat}) FROM Games WHERE AwayTeam = '{AwayTeam}' AND League = '{League}' AND {stat} IS NOT NULL AND date < '{date}'"
                    away_avr = get_value(away_avr,)
                    away_avr = round(away_avr, 2)
                    
                    away_stats[stat] = away_avr

                    

                for stat in stats_list:
                #Goals scoerd
                    league_avr = f"SELECT AVG({stat}) FROM Games WHERE League = '{League}' AND {stat} IS NOT NULL AND date < '{date}'"
                    league_avr = get_value(league_avr,)
                    league_avr = round(league_avr, 2)
                    
                    league_stats[stat] = league_avr

                Home_xStats = {}
                Away_xStats = {}

                for stat in home_stats_list:
                    
                    home_att_str = home_stats[stat] / league_stats[stat]
                    away_def_str = away_stats[stat] / league_stats[stat]
                    league_avr = league_stats[stat]
                    xStat = home_att_str * away_def_str * league_avr

                    xstat = round(home_att_str * away_def_str * league_avr ,2)
                    Home_xStats[stat] = xstat

                    
                    

                for stat in away_stats_list:
                    
                    away_att_str = away_stats[stat] / league_stats[stat]
                    home_def_str = home_stats[stat] / league_stats[stat]
                    league_avr = league_stats[stat]
                    xStat = home_att_str * away_def_str * league_avr

                    xstat = round(away_att_str * home_def_str * league_avr ,2)
                    Away_xStats[stat] = xstat
        

                query = f"""
                INSERT INTO Expected_Games (match_id, date, League, Country, HGames, AGames, HomeTeam, AwayTeam, HGoals, AGoals, xHomePoss, xAwayPoss, xHomeShots, xAwayShots, xHomeSonT, xAwaySonT, xHomeSoffT, xAwaySoffT, xHomeBS, xAwayBS, xHomeCor, xAwayCor, xHomeOff, xAwayOff, xHomeFoul, xAwayFoul, xHomeYellow, xAwayYellow, xHomeRed, xAwayRed, xHomePass, xAwayPass, xHomeAccPass, xAwayAccPass, xHomePassOff, xAwayPassOff, xHomeAccLongB, xAwayAccLongB, xHomeAccLongBpercent, xAwayAccLongBpercent, xHomeAccCross, xAwayAccCross, xHomeAccCrosspercent, xAwayAccCrosspercent, xHomeSuccDribb, xAwaySuccDribb, xHomeSuccDribbpercent, xAwaySuccDribbpercent, xHomeDuelsW, xAwayDuelsW, xHomeTackW, xAwayTackW, xHomeTackWpercent, xAwayTackWpercent, xHomeInt, xAwayInt, xHomeClear, xAwayClear)
                VALUES ({match_id}, '{date}', '{League}', '{Country}', {HGames}, {AGames}, '{HomeTeam}', '{AwayTeam}', {HGoals}, {AGoals}, {Home_xStats['HomePoss']}, {round(100 - Home_xStats['HomePoss'],2)}, {Home_xStats['HomeShots']}, {Away_xStats['AwayShots']}, {Home_xStats['HomeSonT']}, {Away_xStats['AwaySonT']}, {round(Home_xStats['HomeShots'] - Home_xStats['HomeSonT'],2)}, {round(Away_xStats['AwayShots'] - Away_xStats['AwaySonT'],2)}, {Home_xStats['HomeBS']}, {Away_xStats['AwayBS']}, {Home_xStats['HomeCor']}, {Away_xStats['AwayCor']}, {Home_xStats['HomeOff']}, {Away_xStats['AwayOff']}, {Home_xStats['HomeFoul']}, {Away_xStats['AwayFoul']}, {Home_xStats['HomeYellow']}, {Away_xStats['AwayYellow']}, {Home_xStats['HomeRed']}, {Away_xStats['AwayRed']}, {Home_xStats['HomePass']}, {Away_xStats['AwayPass']}, {Home_xStats['HomeAccPass']}, {Away_xStats['AwayAccPass']}, {Home_xStats['HomePassOff']}, {Away_xStats['AwayPassOff']}, {Home_xStats['HomeAccLongB']}, {Away_xStats['AwayAccLongB']}, {Home_xStats['HomeAccLongBpercent']}, {Away_xStats['AwayAccLongBpercent']}, {Home_xStats['HomeAccCross']}, {Away_xStats['AwayAccCross']}, {Home_xStats['HomeAccCrosspercent']}, {Away_xStats['AwayAccCrosspercent']}, {Home_xStats['HomeSuccDribb']}, {Away_xStats['AwaySuccDribb']}, {Home_xStats['HomeSuccDribbpercent']}, {Away_xStats['AwaySuccDribbpercent']}, {Home_xStats['HomeDuelsW']}, {Away_xStats['AwayDuelsW']}, {Home_xStats['HomeTackW']}, {Away_xStats['AwayTackW']}, {Home_xStats['HomeTackWpercent']}, {Away_xStats['AwayTackWpercent']}, {Home_xStats['HomeInt']}, {Away_xStats['AwayInt']}, {Home_xStats['HomeClear']}, {Away_xStats['AwayClear']})
                """
                call_db(query)

                print("Game Added to DB!", i, HomeTeam, AwayTeam)
                i = i + 1
        except:
            print("Game not Added!", i, HomeTeam, AwayTeam)
            i = i + 1 
    





Game Added to DB! 1 Häcken IFK Göteborg
Game Added to DB! 2 Sirius Malmö FF
Game Added to DB! 3 Falkenbergs FF IFK Norrköping
Game Added to DB! 4 Aalesund Start
Game Added to DB! 5 Sarpsborg 08 FK Haugesund
Game Added to DB! 6 Djurgården Häcken
Game Added to DB! 7 Örebro Falkenbergs FF
Game Added to DB! 8 IFK Göteborg Malmö FF
Game Added to DB! 9 Varbergs BoIS FC Östersunds FK
Game Added to DB! 10 Helsingborg Hammarby
Game Added to DB! 11 Odds Ballklubb Rosenborg
Game Added to DB! 12 Sarpsborg 08 Aalesund
Game Added to DB! 13 FK Haugesund Stabæk
Game Added to DB! 14 Bodø/Glimt Strømsgodset
Game Added to DB! 15 Elfsborg Sirius
Game Added to DB! 16 Hammarby Falkenbergs FF
Game Added to DB! 17 Varbergs BoIS FC Djurgården
Game Added to DB! 18 Malmö FF Helsingborg
Game Added to DB! 19 Molde Start
Game Added to DB! 20 AIK Elfsborg
Game Added to DB! 21 Kalmar FF Örebro
Game Added to DB! 22 IFK Göteborg Östersunds FK
Game Added to DB! 23 Mjällby Sirius
Game Added to DB! 24 Häcken IFK Norrköpin